# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = '1b944a9b-fdae-4f97-aeb1-b7eea0beac53'
resource_group = 'aml-quickstarts-134449'
workspace_name = 'quick-starts-ws-134449'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='heart')
dataset.to_pandas_dataframe()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code A2WAMM5L4 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9,Column10,Column11,Column12,Column13
0,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
1,75,0,582,0,20,1,265000,1.9,130,1,0,4,1
2,55,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
3,65,0,146,0,20,0,162000,1.3,129,1,1,7,1
4,50,1,111,0,20,0,210000,1.9,137,1,0,7,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,62,0,61,1,38,1,155000,1.1,143,1,1,270,0
296,55,0,1820,0,38,0,270000,1.2,139,0,0,271,0
297,45,0,2060,1,60,0,742000,0.8,138,0,0,278,0
298,45,0,2413,0,38,0,140000,1.4,140,1,1,280,0


In [2]:
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive.parameter_expressions import choice
import os

import joblib

from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

from azureml.core.webservice import LocalWebservice, AciWebservice
from azureml.core.model import Model
import requests
import json

from azureml.core import ScriptRunConfig

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config()
experiment_name = 'ml-experiment-2'

hyper_experiment=Experiment(ws, experiment_name)

hyper_experiment

Name,Workspace,Report Page,Docs Page
ml-experiment-2,quick-starts-ws-134449,Link to Azure Machine Learning studio,Link to Documentation


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [4]:
# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster

amlcompute_cluster_name = "notebook134449"

# Verify that cluster does not exist already

try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)


Found existing cluster, use it.

Running


In [6]:
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
    '--C': choice(0.5, 1.5),
    'max_iter': choice( 50, 150)
})

#TODO: Create your estimator and hyperdrive config
# estimator = SKLearn(
#     source_directory=".",
#     compute_target=amlcompute_cluster_name,
#     entry_script="train.py",
#     script_params={'--C':1.0, '--max_iter':100})

# Create the environment
sklearn_env = Environment.get(workspace=ws, name="AzureML-Tutorial")

src = ScriptRunConfig(source_directory='./',
                      script='train.py',
                      compute_target = compute_target,
                      environment=sklearn_env)
                    

if "training" not in os.listdir():
    os.mkdir("./training")

if "models" not in os.listdir():
    os.mkdir("./models")

hyperdrive_run_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling=param_sampling,
    policy=early_termination_policy,
    primary_metric_name = "Accuracy",
    primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
    max_total_runs = 100,
    max_concurrent_runs = 2)    


In [7]:
#TODO: Submit your experiment
hyperdrive_run = hyper_experiment.submit(config=hyperdrive_run_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_87b0d36d-3f8d-416f-a7b5-2f14eede7f82
Web View: https://ml.azure.com/experiments/ml-experiment-2/runs/HD_87b0d36d-3f8d-416f-a7b5-2f14eede7f82?wsid=/subscriptions/1b944a9b-fdae-4f97-aeb1-b7eea0beac53/resourcegroups/aml-quickstarts-134449/workspaces/quick-starts-ws-134449

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-01-12T12:58:48.596772][API][INFO]Experiment created<END>\n""<START>[2021-01-12T12:58:49.097669][GENERATOR][INFO]Trying to sample '2' jobs from the hyperparameter space<END>\n""<START>[2021-01-12T12:58:49.253511][GENERATOR][INFO]Successfully sampled '2' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2021-01-12T12:58:49.8783456Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END><START>[2021-01-12T12:59:20.3483545Z][SCHEDULER][INFO]The execution environment was successfully prepared.<END><START>[2021-01-12T12:59:20.3510415Z][SCHEDULER][INFO]Scheduling job, id='HD_87b0d36d

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "User errors were found in at least one of the child runs.",
        "messageParameters": {},
        "details": []
    },
    "time": "0001-01-01T00:00:00.000Z"
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Activity Failed:\n{\n    \"error\": {\n        \"code\": \"UserError\",\n        \"message\": \"User errors were found in at least one of the child runs.\",\n        \"messageParameters\": {},\n        \"details\": []\n    },\n    \"time\": \"0001-01-01T00:00:00.000Z\"\n}"
    }
}

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service